In [0]:
# set up the notebook and download the data
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME =ahmedaraby
%env KAGGLE_KEY =af9b0168bb30bb0c4ec617dc6e25c015
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip "fcis-sc-deeplearning-competition.zip"

In [8]:
# imports 

import pickle
import os 
from os import listdir
import numpy as np
import pandas as pd  
import cv2 
import matplotlib.pyplot as plt  
import matplotlib.image as mpimg

# tensorflow 
import tensorflow as tf 
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization , batch_normalization 
from tflearn.optimizers import Momentum
from tflearn.layers.estimator import regression
from tflearn.optimizers import SGD
from tflearn.initializations import normal , xavier
from __future__ import division, print_function, absolute_import

# keras 
import keras
from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten
from keras.layers import Conv2D , MaxPooling2D
from keras.models import load_model
from keras.regularizers import l2
from keras.optimizers import SGD , Adam
from keras.initializers import RandomNormal
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [0]:
#### utils 


def save_into_dics(obj , name):
    # name should include the extension
    pickle_out = open(name , 'wb')
    pickle.dump(obj , pickle_out)
    pickle_out.close()
    return


def read_from_dics(name):
    pickle_in = open(name , 'rb')
    obj = pickle.load(pickle_in)
    pickle_in.close()
    return obj


def shuffle(X , Y):
    index = np.random.permutation(len(X))

    X = X[index]
    Y = Y[index]

    return X , Y


In [0]:
# preprocessing 
def subtract_mean(img):
    mean = [132.46343732, 118.07325045, 102.6140029]
    img = img - mean 
    img = normalize_min_max(img)
    return img

def normalize_min_max(img):
    min =  np.min(img)
    max = np.max(img)
    img = (img - min) / (max-min)  
    return img

def resize_image(img , inputH = 256 , inputW = 256 , inputC = 3):
    H , W , _ = img.shape

    # resize the shorter side
    # handle very small images 
    toH = inputH 
    toW = inputW

    if W >= H and W>=inputH:
        toW = W
    if H >= W and H>=inputW:
        toH = H

    resized_img = cv2.resize(img, dsize=(toH, toW))
    # resized_img = resized_img.reshape(toH , toW , inputC)  # big massive bug   **********************************************

    # get the start/ end points of the center 256*256
    H , W  , _ = resized_img.shape

    midH = int(H/2)
    midW = int(W/2)

    # the % thing is to handle non even windows 
    startH = midH - inputH//2
    endH = midH + inputH//2 + (inputH%2)  # excluded

    startW  = midW - inputW//2
    endW = midW + inputW//2 + (inputW%2)

    # crop center inputH * inputW
    croped_img = resized_img[startH:endH , startW:endW , :]

    H , W , C = croped_img.shape
    assert (H==inputH and W == inputW and C== inputC)
    return croped_img


def preprocessing(img  , inputH=256 , inputW=256 , inputC=3):
    img = resize_image(img , inputH , inputW , inputC)
    return img

In [0]:
##### agumentation 
import imutils  # for rotating without cropping the edges
# for preprocessing  
inputH = 256
inputW = 256
inputC = 3

# to be feed for the CNN
CNNH = 227
CNNW = 227
CNNC = 3

mode = -1 # -1 BGR , 0  gray

def horizontal_reflication_img(img , CNNH=227 , CNNW=227 , CNNC=3):
    H, W, C = img.shape
    assert (H == CNNH and W == CNNW and C == CNNC)
    hflip = cv2.flip(img , 1)
    return hflip

def extract_random_portion(img , CNNH = 227, CNNW = 227, CNNC = 3):
    H , W , C = img.shape
    assert (H==inputH and W==inputW and C==inputC)

    startH = np.random.randint(low=0 , high=H - CNNH)
    startW = np.random.randint(low=0 , high=W - CNNW)
    endH = startH + CNNH
    endW = startW + CNNW

    assert (endH <= H and endW <=W)
    ex_img = img[startH:endH , startW:endW , :]
    H , W , C = ex_img.shape
    assert (H == CNNH and W == CNNW and C == CNNC)
    return ex_img

def rotate_img_keep(img):
    # images get reshaped
    angle = np.random.random_integers(low=-5 , high=5)
    rot_img = imutils.rotate_bound(img , angle)
    # rot_img = imutils.rotate(img , angle)
    return rot_img

def add_gauusian_noise(img):
    blured_img = cv2.GaussianBlur(img , (7 , 7) , 1)
    return blured_img

def add_salt_peper_noise(img):
    salty_img = np.copy(img)
    for i in range(0 , img.shape[0]):
        for j in range(0 ,  img.shape[1]):
            rand = np.random.random_sample()
            if rand <= 0.005:
                salty_img[i , j , :] = 0
            elif rand <= 0.009:
                salty_img[i, j, :] = 255
    return salty_img

def Agumentation(img,num, CNNH=227 , CNNW=227 , CNNC=3):
    ex_img = extract_random_portion(img , CNNH , CNNW , CNNC)
    Hfliped_img = horizontal_reflication_img(ex_img , CNNH , CNNW , CNNC) 

    #rotate_img = rotate_img_keep(ex_img)   #rotate change the size of the image
    #gauss_img = add_gauusian_noise(ex_img)
    #salty_img = add_salt_peper_noise(ex_img)
    # make sure they are 227
    # rotate_img = cv2.resize(img, dsize=(CNNH, CNNW))
    # string is to make the image unique form the original
    img_l = [ ( ex_img  , str(CNNH)+str(num)+".jpg"), ( Hfliped_img , "flip"+str(num)+".jpg")]
             #,(gauss_img , "gauss"+str(num)+".jpg") ]
             #, (salty_img , "salty"+str(num)+".jpg")] 
             #(rotate_img , "rotate"+str(num)+".jpg") 

    # string is to make the image unique form the original
    #img_l = [ ( ex_img  , str(CNNH)), ( Hfliped_img , "flip")]
    return img_l
    
def combine_agumentation(img):
    # **** now we are officially dublicated *** to balance the classes variations 
    #w = np.random.random()
    #img = rotate_img_keep(img)   #rotate change the size of the image
    
    #if w < 0.2:
        #img = add_gauusian_noise(img)
    #if w < 0.5:
      #img = add_salt_peper_noise(img)
    return img

In [0]:
# generate train and validation data 

# labels as in kaggle but 0 - based 
label_dict = {'airport_inside': 0, 'bakery': 1, 'bedroom': 2, 'greenhouse': 3,
              'gym': 4, 'kitchen': 5, 'operating_room': 6, 'poolinside': 7,
              'restaurant': 8, 'toystore': 9}

# for preprocessing  
inputH = 256
inputW = 256
inputC = 3

# to be feed for the CNN
CNNH = 227
CNNW = 227
CNNC = 3

mode = -1 # -1 BGR , 0  gray

# image.shape (height ,  width , channels )

corrupted = ['indooPool_Inside_gif.jpg']

def check_image(img):
    if mode==-1 and ( len(img.shape)!=3 or img.shape[2] != 3): # img.shape[0] < 256 or img.shape[1] < 256 or (we will do upsampling)
        return 0
    return 1


In [0]:

path = '/content/train/train'
save_path_train = '/content/my_train/train'
save_path_valid = '/content/my_valid/valid'
main_file = listdir(path)
os.mkdir("/content/my_valid")
os.mkdir("/content/my_valid/valid")

os.mkdir("/content/my_train")
os.mkdir("/content/my_train/train")
for class_name in main_file:
  new_path_train = os.path.join(save_path_train , class_name)
  new_path_valid = os.path.join(save_path_valid , class_name)
  os.mkdir(new_path_train)
  os.mkdir(new_path_valid)

In [0]:
# making Directories 

Data_path = '/content/train/train'
save_train_path = '/content/my_train/train'
save_valid_path = '/content/my_valid/valid'
save_test_path = '/content/my_test/test'

MAX = 600  # max number of images in each class 
num = 0
def build_Data_Set():
    global num
    train_file = os.listdir(Data_path)

    for class_name in train_file:
        class_file_path = os.path.join(Data_path , class_name)
        class_file = listdir(class_file_path)

        # ***** handle un blanced classes number ******
        class_length = len(class_file)
        cnt = 0
        ACG = MAX - class_length
        tl_img = []
        tl_label = []  # we don't need it 
        for img_name in class_file:
            img_path = os.path.join(class_file_path , img_name)

            # corrupted images
            if img_name in corrupted:
                continue
            img = cv2.imread(img_path, mode) 
            # check img 
            if check_image(img)==0:
                continue

            print("final path :" , img_path)
            # apply preprocessing   (256*256 crop)
            img = preprocessing(img , inputH , inputW , inputC)

            # apply agumentation
            img_l = Agumentation(img ,num, CNNH , CNNW , CNNC)
            num +=1
            # temp save into list 
            
            for q in range(0 , len(img_l) , 1):
              if q!=0 and cnt >= ACG:
                break
              tl_img.append(img_l[q])
              tl_label.append(label_dict[class_name]) # *****
              
            if cnt < ACG:
                cnt += ( len(img_l)-1 )
          

        print("length ###########################" , len(tl_img) , cnt)
        # increase the class that have < MAX images 
        diff = MAX - len(tl_img)
        sz = len(tl_img)
        for i in range(0 , diff):
            ind = np.random.randint(low=0 , high=sz)
            timg = tl_img[ind]
            # apply some agumentation 
            rot_img = combine_agumentation(timg[0])  # ***************************** will change the dimension a bit ************************
            tl_img.append((rot_img , "inc"+timg[1]))   # was BUG 
            tl_label.append(label_dict[class_name])  # ***

        # split into train , valid  10% validation 
        end = len(tl_img) - int(0.1 * len(tl_img))
        index = np.random.permutation(len(tl_img))

        print("length ###########################" , len(tl_img))
        # save into new directories 

        # save into train 
        for i in range(0 , end):
          print("saving train")
          ind = index[i]
          timg = tl_img[ind]
          tmp_class_path =os.path.join(save_train_path , class_name) 
          tmp_img_path = os.path.join(tmp_class_path , timg[1])
          cv2.imwrite(tmp_img_path , timg[0])
        
        # save into valid 
        for i in range(end , len(tl_img)):
          print("saving valid")
          ind = index[i]
          timg = tl_img[ind]
          tmp_class_path =os.path.join(save_valid_path , class_name)
          tmp_img_path = os.path.join(tmp_class_path , timg[1])
          cv2.imwrite(tmp_img_path , timg[0])

print("start")
build_Data_Set()
print("end")

In [0]:
# generate train and validation data   csv files 

# labels as in kaggle but 0 - based 
label_dict = {'airport_inside': 0, 'bakery': 1, 'bedroom': 2, 'greenhouse': 3,
              'gym': 4, 'kitchen': 5, 'operating_room': 6, 'poolinside': 7,
              'restaurant': 8, 'toystore': 9}
    
def generate_csv_file(path):
    class_file = listdir(path)
    
    paths = []
    labels = []

    for class_name in class_file:
      class_path = os.path.join(path , class_name)
      class_file = listdir(class_path)

      for img_name  in class_file:
          img_path = os.path.join(class_path , img_name)

          # save the path and the id 
          #label_id = label_dict[class_name]  # 0 - based as kaggle 
          paths.append(img_path)
          labels.append(class_name)
    
    paths , labels = shuffle(np.array(paths) , np.array(labels))
    print("length os the whole train data is " , len(paths) , len(labels))
    save_to_csv(paths , labels , 'train.csv')

def save_to_csv(image_path, labelid, name):
    tmp_dict = {"path":image_path , "label":labelid}
    Data = pd.DataFrame(tmp_dict)
    Data.to_csv(name)
    return Data

print("start")
generate_csv_file("/content/my_train/train")
print("end")

In [0]:

def VGG_16(lmda=0.0005 , drop=0.3):

    # winit = RandomNormal(mean=0.0 , stddev=0.01)

    model = Sequential()

    # first block of conv layers
    model.add(Conv2D(64 , (3 , 3) , padding='same' , activation='relu' , input_shape=(227 , 227 , 3) , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(64 , (3 , 3) , strides=(1 , 1) , padding='same'  , activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # second block of conv layers
    model.add(Conv2D(128 , (3 , 3) , padding='same' , activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # third block of conv layers
    model.add(Conv2D(256 , (3 , 3) , padding='same' , activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(256 , (3 , 3) , padding='same' , activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # forth block
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(lmda)))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # fifth block
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(lmda)))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

    # flatten
    model.add(Flatten())  # channel last
    #model.add(Dropout(rate=drop))           # ************************************************************

    # fully connected layers
    model.add(Dense(4096 , activation='relu' , kernel_regularizer=l2(lmda)))
    model.add(Dropout(rate=drop))

    model.add(Dense(4096, activation='relu' , kernel_regularizer=l2(lmda)  ))
    model.add(Dropout(rate=drop))

    model.add(Dense(10, activation='softmax',  kernel_regularizer=l2(lmda)  ))

    return model

def freeze_layers(model):
    print("freze conv layers")
    index = [ 0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15]   #, 16]
    for i in range(0 , len(index)):
        model.layers[index[i]].trainable=False   # bug *****************************

    #print("freze FC layers")
    #index = [19, 21, 23]
    #for i in range(0, len(index)-1):
      #model.layers[index[i]].trainable = False    # bug ********************
      
    return model

# load trianable parameters 

def set_conv_weights(model, index , path="/content/drive/My"+" "+"Drive/vgg16_weights.npz"):
    print("initialize  some conv layers ")
    W = np.load(path)
    block = 1
    layer = 1

    for i in range(0 , len(index)):
        # reset the pattern , by detecting
        # the max pool layer and flatten layer
        if i!=0 and index[i]-index[i-1]>1:
            block += 1
            layer = 1

        weight = "conv" + str(block) + '_' + str(layer) + '_' + 'W'
        bias = "conv" + str(block) + '_' + str(layer) + '_' + 'b'

        # make sure that we are the right match
        X , Y =model.layers[index[i]].get_weights()
        X = X.shape
        Y = Y.shape
        XX = W[weight].shape
        YY = W[bias].shape
        #print(X , XX , Y , YY)
        assert(X == XX and Y == YY)

        #print("for debugining " , index[i] , weight , bias)
        model.layers[ index[i] ].set_weights([W[weight] , W[bias]])
        layer +=1


    return model


def set_fc_weights(model , index, path="/content/drive/My"+" "+"Drive/vgg16_weights.npz"):
    print(" initialize some FC layers ")
    W = np.load(path)

    layer = 6
    for i in range(0 , len(index)-1): # leave the output layer
        model.layers[index[i]].set_weights([W["fc"+str(layer)+"_W"] , W["fc"+str(layer)+"_b"]])
        layer += 1

    return model

def set_weights(model):
    # I think we don't need to return the model back as we edit what in it not assign to it 
    print("in set weights")
    # conv layers
    index = [0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15 , 16]
    model = set_conv_weights(model, index)

    # fulle connected layers
    index = [19, 21, 23]
    model = set_fc_weights(model , index)
    return model



In [0]:
# VGG-16   keras from scratch

print("start")
Batch_Size = 256
Learning_rate = 0.001
model = VGG_16() 
model = freeze_layers(model)
model = set_weights(model)

sgd = SGD(lr=Learning_rate , momentum=0.9)
# opt = Adam(lr=0.01)
model.compile(optimizer=sgd , loss="categorical_crossentropy" , metrics=["accuracy"])
print(model.summary())

# callbacks 
best_val_acc = ModelCheckpoint('bestmodel.h5' , monitor='val_acc' , save_best_only=True , mode='max')



# generators  Image Agumentation
train_datagen = ImageDataGenerator(
                               data_format='channels_last')
valid_datagen = ImageDataGenerator(
                               data_format='channels_last')

# Data generator 
train_generator = train_datagen.flow_from_directory(directory='/content/my_train/train',
                                                    target_size=(227, 227),
                                                    batch_size=Batch_Size,
                                                    shuffle=True,
                                                    class_mode='categorical',
                                                    )

valid_generator = valid_datagen.flow_from_directory(directory='/content/my_valid/valid',
                                                    target_size=(227, 227),
                                                    batch_size=Batch_Size,
                                                    shuffle=True,
                                                    class_mode='categorical',
                                                    )


# train the model 
model.fit_generator(train_generator,
                    steps_per_epoch = 5893 // Batch_Size,
                    epochs=100 ,
                    validation_data=valid_generator, 
                    callbacks=[best_val_acc], 
                    validation_steps = 1113 // Batch_Size)

print("******** mapping *********")

print(train_generator.class_indices)

# model.fit(x=X_train , y=Y_train , batch_size=256 , epochs=50 ,callbacks=[best_val_acc], validation_data=(X_valid , Y_valid))

print("save weights")
model.save_weights('full_weights.h5')

print("save model")
model.save('full_model.h5')
print("end")

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!rm bestmodel.h5 
!rm full_model.h5
!rm full_weights.h5
!rm adc.json

In [0]:
# predict 
def get_5_portions(img , inputH=227 , inputW=227):
    H , W , _ = img.shape

    # first portion
    img_l = []
    img_l.append(img[0:227 , 0:227 , :])

    # second portion
    img_l.append(img[0:227 , W-228:W-1 , :])  # order of the second index matters !!!

    # third portion
    img_l.append(img[H-228:H-1 , 0:227 , :])

    # 4th portion
    img_l.append(img[H-228:H-1 , W-228:W-1 , :])

    # get center portion
    midH = H//2
    midW = W//2

    startH = midH - inputH//2
    endH = midH + inputH//2 + (inputH%2)

    startW = midW - inputW//2
    endW = midW + inputW // 2 + (inputW % 2)
    img_l.append(img[startH:endH , startW:endW , :])

    # check
    """
    index = 0
    for timg in img_l:
        H , W , C = timg.shape
        # print(index)  # for debuging
        assert(H==inputH and W==inputW and C==3)
        index +=1
    """
    return img_l

def predict(X):
    labels = model.predict(X)
    labels = np.squeeze(np.argmax(labels , axis=1))

    labels = np.bincount(labels)
    labels= np.argmax(labels)
    return labels

def test(path):
    img_name  =  []
    img_label =  []

    mean = np.load('DS_local_mean.npy')
    mean = mean.reshape(1 , 1 , 3)

    file = listdir(path)
    for name in file:
        final_path = path + "/" + name #os.path.join(path , name)
        img = cv2.imread(final_path , -1)

        ###################################################################################

        # bad image
        if name=='1412_mb_file_0a8c5_gif.jpg':   # gif image
            print("bad image ")
            img_name.append(name)
            img_label.append(rlabel_dict['greenhouse'])
            continue
        if name=="operating_room_07_06_altavista.jpg":  # gray scale img and my model take only RGB
            print("bad image ")
            img_name.append(name)
            img_label.append(rlabel_dict['operating_room'])
            continue

        ####################################################################################

        # put the image in the same conditions of training
        print(name)
        # RGB
        img = img[: , : , (2 , 1 , 0)]

        # resize into 256 * 256
        img = preprocessing(img)

        # subtract the mean
        img = img - mean

        # normalize the image
        img = normalize_min_max(img)

        # this procedure is from alexnet - image net paper
        # generate all the agumented imgs and vote for label
        img_l = []
        img_l = get_5_portions(img)
        #get the flips of each image
        sz = len(img_l)
        for i in range(0 , sz):
            fimg= horizontal_reflication_img(img_l[i])
            img_l.append(fimg)
        img_l = np.array(img_l).reshape(-1 , 227 , 227 , 3)  # to be predicted
        label = predict(img_l)

        # build the file
        img_name.append(name)
        img_label.append(label+1)  # stupid file
    return img_name , img_label  # to be saved into csv file

def save_to_csv_file(names , labels):
  Data = {"id":names , "lable":labels}
  Data = pd.DataFrame(Data)
  Data.to_csv("mysubmit.csv")
  return Data


print("start")

model = load_model("/content/full_model.h5")#"/content/drive/My"+" "+"Drive/model.h5") # this is model with only output trained
rlabel_dict = read_from_dics('rlabel_dict')

print(rlabel_dict)

names , labels = test('/content/test/test')
print("\n\n")
print(names)
print(labels)

print("end")


In [0]:
print(os.listdir("/content/train/train/"))
colab_l_i = read_from_dics("label_dict")
print(colab_l_i)


In [0]:
# mount google drive with colab
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# doawnload model wights 
from google.colab import files
files.download("checkpoint")
files.download("model.tfl.meta")
files.download("model.tfl.index")
files.download("model.tfl.data-00000-of-00001")

In [0]:

print("start")
# load data
X = np.load('/content/X.npy')
Y = np.load('/content/Y.npy')
sess = tf.Session()
Y = sess.run(tf.one_hot(Y , depth=10))
sess.close()
Y = np.squeeze(Y)

model = load_model("/content/drive/My"+" "+"Drive/model.h5")
#print(model.summary())
pred = model.predict(X)
print(model.evaluate(X , Y))

print(pred.shape)

In [0]:
!rm X.npy
!rm Y.npy

In [0]:
# play around a little 
X = np.load("X.npy")
img = X[55]
print(img)
plt.imshow(img)

In [0]:
import keras
print(keras.__version__)

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Save Keras Model or weights on google drive

# create on Colab directory

model_file = drive.CreateFile({'title' : 'bestmodel94_delaug_no_norm.h5'})
model_file.SetContentFile('bestmodel94_delaug_no_norm.h5')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

In [0]:
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5